In [1]:
# basics
import pandas as pd
import numpy as np
import time
import pickle 

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch_directml

# model
from scripts.classes import Botzee, BotzeeDataset

# testing
from scripts.functions import play_botzee, model_pick_dice, model_pick_score

In [2]:
gpu = torch_directml.device()
use_gpu = False

batch_size = 130
n_epochs = 10 # 0

## Data Prep

In [3]:
training_df = pd.read_csv('data/training_df.csv')

In [4]:
X_cols = [
    x for x in training_df.columns if
    x != 'score_pick' and
    x != 'turn_score' and
    x != 'post_total_score'
]

y1_cols = [x for x in training_df.columns if 'picks_1' in x]
y2_cols = [x for x in training_df.columns if 'picks_2' in x]
y3_cols = [x for x in training_df.columns if x == 'score_pick']

X = training_df[X_cols]
X = X.fillna(-1)

y1 = training_df[y1_cols]
y2 = training_df[y2_cols]
y3 = pd.get_dummies(training_df[y3_cols]).astype(int)

In [5]:
X.columns

Index(['game', 'turn', 'pre_total_score', 'chance_score', 'ones_score',
       'twos_score', 'threes_score', 'fours_score', 'fives_score',
       'sixes_score', 'three_kind_score', 'four_kind_score',
       'full_house_score', 'small_straight_score', 'large_straight_score',
       'yahtzee_score', 'hand_1_dice_1', 'hand_1_dice_2', 'hand_1_dice_3',
       'hand_1_dice_4', 'hand_1_dice_5', 'chance_potential_1',
       'ones_potential_1', 'twos_potential_1', 'threes_potential_1',
       'fours_potential_1', 'fives_potential_1', 'sixes_potential_1',
       'three_kind_potential_1', 'four_kind_potential_1',
       'full_house_potential_1', 'small_straight_potential_1',
       'large_straight_potential_1', 'yahtzee_potential_1', 'picks_1_dice_1',
       'picks_1_dice_2', 'picks_1_dice_3', 'picks_1_dice_4', 'picks_1_dice_5',
       'hand_2_dice_1', 'hand_2_dice_2', 'hand_2_dice_3', 'hand_2_dice_4',
       'hand_2_dice_5', 'chance_potential_2', 'ones_potential_2',
       'twos_potential_2', 't

In [6]:
y1.columns

Index(['picks_1_dice_1', 'picks_1_dice_2', 'picks_1_dice_3', 'picks_1_dice_4',
       'picks_1_dice_5'],
      dtype='object')

In [7]:
y2.columns

Index(['picks_2_dice_1', 'picks_2_dice_2', 'picks_2_dice_3', 'picks_2_dice_4',
       'picks_2_dice_5'],
      dtype='object')

In [8]:
y3.columns

Index(['score_pick_chance', 'score_pick_fives', 'score_pick_four_kind',
       'score_pick_fours', 'score_pick_full_house',
       'score_pick_large_straight', 'score_pick_ones', 'score_pick_sixes',
       'score_pick_small_straight', 'score_pick_three_kind',
       'score_pick_threes', 'score_pick_twos', 'score_pick_yahtzee'],
      dtype='object')

## Model Setup

In [9]:
# define ranges for masks
indices_branch1 = [x for x in range(list(X.columns).index('picks_1_dice_1'))]
indices_branch2 = [x for x in range(list(X.columns).index('picks_2_dice_1'))]
total_features = X.shape[1]

# create masks
mask1 = torch.zeros(total_features, dtype=torch.bool)
mask1[indices_branch1] = True

mask2 = torch.zeros(total_features, dtype=torch.bool)
mask2[indices_branch2] = True

# combine masks
masks = [mask1, mask2]

### Instantiate Model and Dataloader, Set Params

In [10]:
# model params
input_sizes = [
    len([x for x in masks[0] if x == True]),
    len([x for x in masks[1] if x == True]),
    X.shape[1]
]
lstm_sizes = input_sizes.copy()

# instantiate model
model = Botzee(input_sizes = input_sizes, lstm_sizes = lstm_sizes, dice_output_size = 5, score_output_size = 13, masks = masks)

# create the dataset and define DataLoader
dataset = BotzeeDataset(X, y1, y2, y3)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = False)

# choose optimizer and loss functions
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fn1 = nn.BCEWithLogitsLoss() # BCELoss() # Binary Cross Entropy for first two branches
loss_fn2 = nn.CrossEntropyLoss()  # Cross Entropy for third branch

### Train Model

In [11]:
model.train()

Botzee(
  (lstm1): LSTM(34, 34)
  (fc1): Linear(in_features=34, out_features=34, bias=True)
  (branch1): Linear(in_features=34, out_features=5, bias=True)
  (lstm2): LSTM(57, 57)
  (fc2): Linear(in_features=57, out_features=57, bias=True)
  (branch2): Linear(in_features=57, out_features=5, bias=True)
  (lstm3): LSTM(80, 80)
  (fc3): Linear(in_features=80, out_features=80, bias=True)
  (branch3): Linear(in_features=80, out_features=13, bias=True)
)

In [12]:
start_time = time.perf_counter()

if use_gpu:
    model.to(gpu)

for epoch in range(1,n_epochs+1):

    for X_batch, Y_batch in dataloader:

        # move to GPU
        if use_gpu:
            X_batch = X_batch.to(gpu)
            Y_batch = [y.to(gpu) for y in Y_batch]

        # Forward pass
        out1, out2, out3 = model.forward(X_batch)

        # Calculate loss for each branch
        loss1 = loss_fn1(out1, Y_batch[0])
        loss2 = loss_fn1(out2, Y_batch[1])
        loss3 = loss_fn2(out3, Y_batch[2])

        # Total loss
        total_loss = loss1 + loss2 + loss3

        # Backward pass and optimize
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # print('Batch done')

    if epoch % (n_epochs/10) == 0:
        elapsed_time = time.perf_counter() - start_time
        print(
            f'Epoch [{epoch}] - Time elapsed: {elapsed_time/60:.2f} min\n'
            f'Total Loss: {total_loss.item():.4f} - L1: {loss1.item():.4f} - '
            f'L2: {loss2.item():.4f} - L3: {loss3.item():.4f}'
        )

Epoch [1] - Time elapsed: 0.03 min
Total Loss: 3.4556 - L1: 0.6725 - L2: 0.5616 - L3: 2.2215
Epoch [2] - Time elapsed: 0.07 min
Total Loss: 3.3521 - L1: 0.6679 - L2: 0.5597 - L3: 2.1245
Epoch [3] - Time elapsed: 0.10 min
Total Loss: 3.2358 - L1: 0.6569 - L2: 0.5502 - L3: 2.0288
Epoch [4] - Time elapsed: 0.14 min
Total Loss: 3.1274 - L1: 0.6460 - L2: 0.5328 - L3: 1.9485
Epoch [5] - Time elapsed: 0.17 min
Total Loss: 3.0920 - L1: 0.6430 - L2: 0.5259 - L3: 1.9231
Epoch [6] - Time elapsed: 0.21 min
Total Loss: 3.0860 - L1: 0.6419 - L2: 0.5179 - L3: 1.9261
Epoch [7] - Time elapsed: 0.24 min
Total Loss: 3.0536 - L1: 0.6385 - L2: 0.5169 - L3: 1.8982
Epoch [8] - Time elapsed: 0.28 min
Total Loss: 3.0262 - L1: 0.6387 - L2: 0.5101 - L3: 1.8774
Epoch [9] - Time elapsed: 0.31 min
Total Loss: 3.0212 - L1: 0.6371 - L2: 0.5058 - L3: 1.8784
Epoch [10] - Time elapsed: 0.35 min
Total Loss: 3.0174 - L1: 0.6360 - L2: 0.5033 - L3: 1.8781


### Test Predictions

In [13]:
# create data for prediction
y_pred = X_batch[0:5,:].clone().detach()
# y_pred = y_pred.unsqueeze(0)
if use_gpu:
    y_pred.to(gpu)

# switch model to eval mode
model.eval()

# make predictions
with torch.no_grad():
    predictions = model(y_pred)

dice_picks_1 = []
for pick in predictions[0][0]:
    if pick >= 0.5:
        dice_picks_1.append(1)
    else:
        dice_picks_1.append(0)

dice_picks_2 = []
for pick in predictions[1][0]:
    if pick >= 0.5:
        dice_picks_2.append(1)
    else:
        dice_picks_2.append(0)

score_pick = predictions[2][0].argmax()

print(
    f'Picks after first roll: {dice_picks_1}\n',
    f'Picks after second roll: {dice_picks_2}\n',
    f'Score Pick: {score_pick}'
)

Picks after first roll: [0, 0, 0, 0, 0]
 Picks after second roll: [0, 1, 1, 1, 1]
 Score Pick: 0


In [14]:
y_pred.shape

torch.Size([5, 80])

In [15]:
play_botzee(1, model_pick_dice, model_pick_score, model = model)

[{'game': 1,
  'turn': 1,
  'pre_total_score': 0,
  'chance_score': -1,
  'ones_score': -1,
  'twos_score': -1,
  'threes_score': -1,
  'fours_score': -1,
  'fives_score': -1,
  'sixes_score': -1,
  'three_kind_score': -1,
  'four_kind_score': -1,
  'full_house_score': -1,
  'small_straight_score': -1,
  'large_straight_score': -1,
  'yahtzee_score': -1,
  'hand_1_dice_1': 6,
  'hand_1_dice_2': 6,
  'hand_1_dice_3': 6,
  'hand_1_dice_4': 4,
  'hand_1_dice_5': 2,
  'chance_potential_1': 24,
  'ones_potential_1': 0,
  'twos_potential_1': 2,
  'threes_potential_1': 0,
  'fours_potential_1': 4,
  'fives_potential_1': 0,
  'sixes_potential_1': 18,
  'three_kind_potential_1': 24,
  'four_kind_potential_1': 0,
  'full_house_potential_1': 0,
  'small_straight_potential_1': 0,
  'large_straight_potential_1': 0,
  'yahtzee_potential_1': 0,
  'picks_1_dice_1': 0,
  'picks_1_dice_2': 1,
  'picks_1_dice_3': 1,
  'picks_1_dice_4': 1,
  'picks_1_dice_5': 0,
  'hand_2_dice_1': 6,
  'hand_2_dice_2': 6,

In [16]:
game_data = play_botzee(1, model_pick_dice, model_pick_score, model = model)
df = pd.DataFrame(game_data)
df

,game,turn,pre_total_score,chance_score,ones_score,twos_score,threes_score,fours_score,fives_score,sixes_score,...,sixes_potential_3,three_kind_potential_3,four_kind_potential_3,full_house_potential_3,small_straight_potential_3,large_straight_potential_3,yahtzee_potential_3,score_pick,turn_score,post_total_score
0,1,1,0,-1,-1,-1,-1,-1,-1,-1,...,0,21,0,0,0,0,0,four_kind,0,0
1,1,2,0,-1,-1,-1,-1,-1,-1,-1,...,0,18,-1,0,0,0,0,sixes,0,0
2,1,3,0,-1,-1,-1,-1,-1,-1,0,...,-1,15,-1,0,0,0,0,ones,1,1
3,1,4,1,-1,1,-1,-1,-1,-1,0,...,-1,0,-1,0,30,0,0,fours,4,5
4,1,5,5,-1,1,-1,-1,4,-1,0,...,-1,24,-1,0,0,0,0,threes,0,5
5,1,6,5,-1,1,-1,0,4,-1,0,...,-1,13,-1,25,0,0,0,full_house,25,30
6,1,7,30,-1,1,-1,0,4,-1,0,...,-1,0,-1,-1,0,0,0,fives,5,35
7,1,8,35,-1,1,-1,0,4,5,0,...,-1,0,-1,-1,0,0,0,chance,16,51
8,1,9,51,16,1,-1,0,4,5,0,...,-1,0,-1,-1,0,0,0,twos,0,51
9,1,10,51,16,1,0,0,4,5,0,...,-1,23,-1,-1,0,0,0,yahtzee,0,51


In [17]:
df.columns

Index(['game', 'turn', 'pre_total_score', 'chance_score', 'ones_score',
       'twos_score', 'threes_score', 'fours_score', 'fives_score',
       'sixes_score', 'three_kind_score', 'four_kind_score',
       'full_house_score', 'small_straight_score', 'large_straight_score',
       'yahtzee_score', 'hand_1_dice_1', 'hand_1_dice_2', 'hand_1_dice_3',
       'hand_1_dice_4', 'hand_1_dice_5', 'chance_potential_1',
       'ones_potential_1', 'twos_potential_1', 'threes_potential_1',
       'fours_potential_1', 'fives_potential_1', 'sixes_potential_1',
       'three_kind_potential_1', 'four_kind_potential_1',
       'full_house_potential_1', 'small_straight_potential_1',
       'large_straight_potential_1', 'yahtzee_potential_1', 'picks_1_dice_1',
       'picks_1_dice_2', 'picks_1_dice_3', 'picks_1_dice_4', 'picks_1_dice_5',
       'hand_2_dice_1', 'hand_2_dice_2', 'hand_2_dice_3', 'hand_2_dice_4',
       'hand_2_dice_5', 'chance_potential_2', 'ones_potential_2',
       'twos_potential_2', 't

In [18]:
torch.save(model, 'models/botzee_base.pth')

In [19]:
botzee_class_args = {key: value for key, value in model.__dict__.items()}

with open('models/botzee_base_args.pkl', 'wb') as f:
    pickle.dump(botzee_class_args, f)